### Introduction

This project is the result of a short programming assignment completed during the interview process for a quantitative researcher role at a trading firm in 2024. Using theory from stochastic differential equations and statistics we formulate an automated trading strategy based on mean-reversion. The analysis and code was created from scratch in response to the assignment. The task was to devise, visualize and backtest a trading strategy given recent data from cocoa futures markets. 

### Futures Spread Trading

The prices of all contracts in the dataset have shown similar behaviour: after a long period of relative stability, prices rapidly increased to all-time highs and have remained at these levels. We will attempt to devise an automated trading strategy based on mean reversion. Even though the futures price data does not show any signs of mean-reversion, it is possible that futures spreads paint a different picture. A spread is defined as the difference between the prices of two futures contracts. Since we are only given data from cocoa futures contracts, the only relevant spreads are calendar spreads (as opposed to inter-commodity spreads). Calendar spreads are combinations of futures contracts with different expiries, but on the same underlying commodity. The first component is a short or long position in a futures contract with an expiry date in the far future, and the second component is the 'opposite' position with expiry in the near future. For example, if the first component is a long position in a December 2025 cocoa futures contract with market exposure $V$, then the second component may be a short position in a December 2024 cocoa futures contract with market exposure $-V$.

### Theoretical considerations

What has caused the recent price increases? It may help to give an economic interpretation. Firstly, prices of near-dated cocoa futures contracts are potentially more sensitive to 'local' negative events such as supply-chain distruptions or extreme weather events. Secondly, prices of far-dated cocoa futures contracts are potentially more sensitive to a change in the expectations of market participants. In this case, local events may have caused the price increase, but only in near-dated contracts. If these events persist and cause market participants to adapt their expectations, far-dated contract prices could move towards the current high prices of near-dated contracts. This would imply that calendar spreads are currently above their usual value and will move downward. A trading strategy could be to buy far-dated futures contracts and sell near-dated futures contracts in equal proportion. 

Unfortunately, we do not have the time in this assignment to test the validity of the above claims, or to test what kind of complicated mean-reversion can occur in spreads in general markets. We will therefore skip further theoretical considerations, model comparison and model selection, and start by fitting a simple model with mean-reverting behaviour to the data, developing an automated strategy based on this information. 

### Mathematical Modelling of spread prices

An Ornstein-Uhlenbeck (OU) process is the solution $X$ to an equation of the following form:

$$
dX_t = \kappa (\mu - X_t) dt + \sigma dW_t,
$$

where $W$ is standard Brownian motion, $\mu$ is the mean, $\kappa$ is the mean-reversion parameter, and $\sigma$ is the volatility parameter.

The general form of the solution of this stochastic differential equation is

$$
X_t = X_0 e^{-\kappa t} + \mu (1 - e^{-\kappa t}) + \sigma \int_0^t e^{-\kappa (t-s)} dW_s
$$

Note that in the case that $\sigma = 0$, the solution to the equation is a deterministic exponential decay (when $\kappa > 0$) or exponential growth (when $\kappa < 0$). In the case that $\kappa = 0$, the solution is equal to standard Brownian motion scaled by $\sigma$.

This kind of process is widely used in the financial literature to model mean-reversion. We note that the process is Markov, but not a martingale.

In modelling spread prices as an OU process, we are assuming that there is a fixed price towards which spreads converge with a strength that increases as the spread moves further away from the mean, and any movement away from the mean is due to randomness. Obviously, this is not a realistic assumption. We thus expect that our inference process will lead to overfitting.

Still, the OU model is suited for our purposes in some ways: firstly, the expectation and other quantities have known closed-form expressions. Secondly, as the solution to a stochastic differential equation, an OU process can be approximated by a sequence of discrete processes with theoretical guarantees on the convergence. Lastly, it is possible to infer a mean from data even if the mean is outside the range of values that the data takes.

### Overview

In the remainder we will prepare data, fit a discretized version of an OU process to the data, define a trading signal based on the inferred parameters, write code to simulate a trading strategy based on this signal, visualize and discuss the results of this strategy.

### Programming

We use pandas throughout the code, and numpy in parts of the code where repetitive calculations are performed on dataframes. We could also add just-in-time compilation to our code with numba or a similar package and further improve the efficiency, but the sparse data and daily data resolution does not justify the added complexity of doing this. Where possible, we use plotly, making use of interactive graphing. We use python modules to divide our code into modelling, utility, backtesting and main sections. The analysis will be run from a jupyter notebook that calls functions from the main module.

In [3]:
# %%
import sys
sys.path.append('src/futures-assignment')


import pandas as pd
import plotly.graph_objects as go
from numpy.lib.stride_tricks import sliding_window_view

import plotly.io as pio
pio.renderers.default = "notebook_connected"

import models
import utils
import simulation
import main

from main import *
from simulation import BacktestColumns, BacktestParams




### Data Preparation

In our simulation, to avoid the situation in which a futures contract expires while in our posession and to simplify the strategy we will regularly 'roll over' the components of the spread, meaning that we close the two components in the spread and exchange them for new components with expiries further in the future. We will assume that we can take this action at market prices without transaction costs, and hence that this action will not immediately impact the exposure or the profit of the strategy. However, the time series obtained from joining together the relevant spreads from before and after the rollover may have significant jumps in it. This can cause issues as we are inferring parameters of a process that is by definition continuous. There are different ways to fix this issue: firstly, we can 'back-adjust' the separate spreads to make a more continuous time series. In that method we keep the data from the most recent spread in our time series as-is, and adjust the next most recent spread by adding a constant so that the two time series are equal at the roll-over point. This means that the jump is removed, but the training data for the model may increasingly diverge from the true price as we look back in time. In our case, this is a worthwile trade-off, hence we implement this method.

Next, we choose to consider only spreads of a single calendar year since this fits neatly with the given data and is in line with the earlier interpretation of our mean-reversion strategy. Moreover, this choice might help reduce possible seasonal effects on the spread  held by the strategy.

Lastly, we choose to consider only the spreads in which the nearest contract is at least 30 days from expiry, and at most 3 months to expiry. This leaves us with a single time series obtained from the data. We denote the dataframe containing this time series by 'continuous_df' in the code.

In [4]:

dir_path = "data/future_chain_time_series"
qid_map = utils.read_data(
    dir_path="data", file_name="map_qid_to_future_contract", file_ext="csv"
)
qid_map["expiration_date"] = qid_map["expiration_date"].apply(
    utils.convert_to_datetime
)
qid_to_symbol = qid_map.set_index("qid")["symbol"].to_dict()
symbol_to_expiration = qid_map.set_index("symbol")["expiration_date"].to_dict()

combined_df = main._create_combined_dataframe(dir_path, qid_to_symbol)
ratios_df = main._add_spread_prices(combined_df)
continuous_df = main._add_continuous_spread_price(
    ratios_df, symbol_to_expiration
)

continuous_df = continuous_df[list(set(continuous_df.columns) - set(ratios_df.columns))]



### Discretization

Let $X$ be an Ornstein–Uhlenbeck process with parameters $\mu$, $\kappa > 0$, and $\sigma > 0$, and let $T > 0$.  
For a natural number $k$, divide $[0,T]$ into $k$ equally spaced sub-intervals of length $\Delta t := T/k$.  
Define the discrete process $\{X^k_n\}_{n=0}$ by

$$
X^k_{n+1} = a + b\,X^k_{n} + U_{n+1}, \qquad n = 0, \dots, k-1,
$$

where $X^k_0 = X_0$, $a = \mu(1 - e^{-\kappa \Delta t})$, $b = e^{-\kappa \Delta t}$, and $U_i$ are identical independently distributed random variables with normal distribution with mean $0$ and $Var(U_i) = \sigma^2 \frac{1 - e^{-2 \kappa \Delta t}}{2 \kappa}$. Under standard conditions, $X^k$ converges to $X$ as $k \to \infty$ in a suitable topology. 

We note that $(X^k_{n}, X^k_{n+1})$ forms a regression model, so the ordinary least-squares estimators for this model are the maximum likelihood estimators for $a$, $b$ and $\sigma$. Since the $U_i$ are normally distributed, these estimators are unbiased. Since the image of a maximum likelihood estimator is again a maximum likelihood estimator, it follows that our estimators for $\mu$, $\kappa$ and $\sigma$ are maximum likelihood estimators, hence asymptotically unbiased. With this knowledge, we will treat the discrete and continuous process as equivalent for the remainder of the analysis.

### Window Size

To capture local behavior, we introduce a time window in our inference: instead of inferring a single set of parameters for the entire training set, we infer a new set of parameters each day by considering only the data in a fixed window extending backwards from that day. A smaller window size can help in finding mean-reversion earlier, but will reduce the information used in the estimation of the parameters even further. As a trade-off, we use a windows size of 100 days.

### Spread time series

Instead of the absolute difference in futures contracts, we will consider the value $$\log{(S_{n} / S_{f})},$$ where $S_n$ is the price of the near-dated futures contract and $S_f$ the price of the far-dated futures contract. This normalizes the data and makes it possible for $\mu$ to obtain meaningful negative values. 

### Trading Signal

We will formulate a relatively simple trading signal, and use uniform lower and upper exit and entry bounds. The resulting strategy will enter or exit a position only when these bounds are crossed.

A suggested trading signal from the literature is

$$
s = \frac{\sqrt {2 \kappa} ( X_t - \mu )}{\sigma}.
$$

We make use of this signal with the additional constraint that the value of $\kappa$ is sufficiently large, to prevent entry into a position due to noisy estimates. This additional constraint holds only for the entry into a position, to prevent a position being held indefinitely.

In [5]:
fitted_df = main._add_model_parameter_fit(
    continuous_df, column_name="continuous_price", window_size = 100
)
signals_df = main._add_trading_signals(fitted_df)


utils.plot_data_dual_y_axis(
    data=signals_df,
    column1="continuous_price",
    column2="kappa",
    title="OU Process Parameters Over Time",
)
utils.plot_data_dual_y_axis(
    data=pd.DataFrame(
        {
            "mu": signals_df["mu"],
            "kappa": signals_df["kappa"],
            "var": signals_df["var"],
        },
        index=signals_df.index,
    ),
    column1="mu",
    column2="var",
    title="OU Process Parameters Over Time",
)

utils.plot_data_dual_y_axis(
    data=signals_df,
    column1="continuous_price",
    column2="mr_signal",
    title="OU Process Parameters Over Time",
)

### Inference results

As expected, we run into a few issues when fitting the model. The estimates of the mean and reversion parameter are discontinuous and spike on multiple occcasions, and the signal is not defined when $\kappa$ is negative. This problem is not prevented in the inference step as we impose no regularity constraints. The erratic behavior is likely caused by model misspecification. Intuitively, the spread price process moved from a stable state around 0 to a state around -0.04, and finally to a stable state around 0.3. The erratic parameter values seem to occur near the end of a transitions between these states. During these transitions, the inferred noise strength does not change significantly, in contrast to the mean and mean-reversion speed. Intuitively, in these situations the deterministic movement in price is large in comparison to the noise, and the erratic parameter values are likely the result of trying to fit an exponential curve to a straight line.

### Training and testing set

We split the dataset into a training and testing set at a ratio of 80:20. We will select thresholds for the sensitivity of the strategy to the trading signal using based on the training set data, and conclude by commenting on the performance of the resulting strategy on the test set.

In [6]:
training_df, testing_df = utils.split_data(data = signals_df, offset = 100, ratio = 0.8)

In [7]:
cols = BacktestColumns(
    signal_column="mr_signal",
    p1_price_column="near_contract_price",
    r1_price_column="preroll_near_contract_price",
    p2_price_column="far_contract_price",
    r2_price_column="preroll_far_contract_price",
    rollover_column="rollover",
    constraint_column="kappa",
)

# Simulate sensitivities and visualize the results
sensitivities_results = simulation.test_signal_sensitivities(
    data=training_df,
    cols=cols,
    lower_constraint=5,
    upper_constraint=100,
    lower_entry_th_max=0,
    lower_entry_th_min=-4,
    upper_entry_th_max=4,
    upper_entry_th_min=0,
    upper_entry_exit_ratio=0.5,
    lower_entry_exit_ratio=0.5,
    n=15,
)

utils.plot_3d_surface(
    sensitivities_results,
    x_col_name="lower_entry_threshold",
    y_col_name="upper_entry_threshold",
    z_col_name="final_pnl",
)

### Entry thresholds


We expect that setting the entry signals close to 0 will reduce the strategy to one that enters and exits a position in a given calendar spread at random times. So, we manually select thresholds that we deem to be sufficiently far away from 0, ensuring that the strategy is still sensitive enough to the trading signal to enter and exit enough positions during the training period.

In [8]:
params = BacktestParams(
    lower_constraint=3,
    upper_constraint=100,
    lower_entry_threshold=-1,
    lower_exit_threshold=-0.5,
    upper_entry_threshold=1.5,
    upper_exit_threshold=0.75,
    contract_size=10,
)

results_df = simulation.backtest(
    data=training_df,
    cols=cols,
    params=params,
)

main._visualize_strategy(results_df, params, n = 1)

strategy_metrics = models.calculate_strategy_metrics(
    pnl_series=results_df['pnl'],
    extrapolation_ratio=252
)

pd.DataFrame([strategy_metrics.__dict__])

,sharpe_ratio,calmar_ratio,hit_ratio,max_drawdown
0,0.291047,1.484158,0.127273,-418


### Training results

With the selected uniform thresholds, our strategy enters and exits 6 positions during the training period. Notably, the spikes in the trading signal do not cause the strategy to enter a position, due to the additional constraint of a sufficiently large $\kappa$, as intended.


We can now check the performance on the test data.

In [9]:
results_df = simulation.backtest(
    data=testing_df,
    cols=cols,
    params=params,
)

main._visualize_strategy(results_df, params)

strategy_metrics = models.calculate_strategy_metrics(
    pnl_series=results_df['pnl'],
    extrapolation_ratio=252
)

pd.DataFrame([strategy_metrics.__dict__])

,sharpe_ratio,calmar_ratio,hit_ratio,max_drawdown
0,1.289215,3.756172,0.176471,-4053


### Conclusion

The signal in the test period seems to suffer from the same issues as in the training period. In the test case, our strategy enters and exits a single position. Although a small profit was made, the testing period is short and the strategy swung wildly between profit and loss during the time that it held a position. From this it seems likely that the profit was obtained due to random chance. Also, the strategy curiously enters a position when the lower threshold is reached from below, likely due to this missing edge case in the code. In other words, the strategy shows some promise, but the rigidity of the OU process and the simplicity of the algorithm make the strategy unfit for direct use, especially in periods of large price movements. 

Luckily there are a few potential ways to addres these issues. Beyond making use of a completely different model or making the model more flexible, we could improve the reliability by adding statistical information such as goodness-of-fit tests. Additionally, we could improve the inference process by adding regularity constraints or retaining information between inference rounds.